In [9]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import numpy as np
import pandas as pd
from operator import itemgetter, attrgetter
import sys
sys.path.insert(0, '..')
from src.data.Preprocessor import Preprocessor

class Classifier:
    def __init__(self):
        #Array für alle Ergebnisse
        self.ergebnis = []
    
    def train_models(self, X_train, X_test, y_train, y_test, models):
        for self.model in models:
            #-----------------------
            #Knn-Classifier
            #-----------------------
            if self.model == 'knn':
                #Optimalen Knn-Classifier bestimmen
                error = []
                for i in range(1, 40):
                    knn = KNeighborsClassifier(n_neighbors=i)
                    knn.fit(X_train, y_train)
                    pred_i = knn.predict(X_test)
                    error.append(np.mean(pred_i != y_test))

                #Knn-Classifier trainieren
                knnclf = KNeighborsClassifier(n_neighbors=7)
                knnclf.fit(X_train, y_train)

                #Knn-Classifier Akkuranz bestimmen
                score = knnclf.score(X_test,y_test)
                self.ergebnis.append(['knn-classifier', score, knnclf])
            #-----------------------
                
            #-----------------------
            #Decision Tree
            #-----------------------
            elif self.model == 'dt':
                #class_weight gebrauchen für DT und RF

                #Optimalen Decision Tree bestimmen
                #Zu testende Decision Tree Parameter
                dt = DecisionTreeClassifier()
                tree_para = {'criterion':['gini','entropy'],'max_depth':[i for i in range(1,20)], 'min_samples_split':[i for i in range (2,20)]}

                #GridSearchCV 
                grd_clf = GridSearchCV(dt, tree_para, cv=5)
                grd_clf.fit(X_train, y_train)

                #Besten gefundenen Decision Tree übergeben
                dt_clf = grd_clf.best_estimator_

                score = dt_clf.score(X_test,y_test)
                self.ergebnis.append(['decision tree', score, dt_clf])
            #-----------------------

            #-----------------------
            #Random Forest
            #-----------------------
            elif self.model == 'rf':
                #rf = RandomForestClassifier(max_depth=8, criterion="entropy", min_samples_split=9)
                rf = RandomForestClassifier(n_estimators=100)
                rf.fit(X_train,y_train)
                score = rf.score(X_test,y_test)
                self.ergebnis.append(['random forest', score, rf])
            #-----------------------

            #-----------------------
            #Support Vector Machine
            #-----------------------
            elif self.model == 'svm':
                svm = SVC(kernel = 'poly')
                svm.fit(X_train, y_train)
                score = svm.score(X_test,y_test)
                self.ergebnis.append(['support vector machine', score, svm])

        return self.ergebnis

def get_classifier():
    #Daten einlesen
    data = pd.read_excel('../data/raw/U bung kNN Klassifizierung Ecoli.xlsx', sheet_name=0)
    X = data.loc[:,data.columns != "Target"]
    y = data["Target"]

    #Models spezifieren
    models = {"rf":RandomForestClassifier, "dt":DecisionTreeClassifier, "knn":KNeighborsClassifier, "svm":SVC}

    #Daten preprocessen
    X_train, X_test, y_train, y_test = Preprocessor(X, y).get_data()

    #Classifier verwenden
    clf = Classifier()
    resultat = clf.train_models(X_train, X_test, y_train, y_test, models)
    
    #Bestes Ergebnis bestimmen und als Modell verwenden
    print("Bestes Model ist: {} mit einer Akkuranz von {}%".format(sorted(resultat, key=itemgetter(1), reverse=True)[0][0],sorted(resultat, key=itemgetter(1), reverse=True)[0][1]*100))
    return sorted(resultat, key=itemgetter(1), reverse=True)[0][2]


print(get_classifier())



C:\Users\Chris\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
Bestes Model ist: random forest mit einer Akkuranz von 88.05970149253731%
RandomForestClassifier()


In [3]:
sorted(ergebnis, key=itemgetter(1), reverse=True)

NameError: name 'ergebnis' is not defined